**MoE parallelism** 
MoE заключается в распределении экспертов между разными работниками, и каждый работник получает разные пакеты обучающих данных. Для уровня, не связанного с MoE, параллелизм экспертов - это то же самое, что параллелизм данных. На уровне MoE токены в последовательности отправляются работникам, соответствующим их соответствующим экспертам, посредством связи "все ко всем". После завершения вычислений соответствующего эксперта они затем повторно передаются исходному работнику по принципу "все ко всем" и упорядочиваются в исходную последовательность для вычисления следующего уровня. Поскольку модели MoE обычно содержат большое количество экспертов, параллелизм экспертов увеличивается в большей степени с увеличением размера модели, чем параллелизм моделей.

![MoE](../pictures/MoE_parallelism.png)

**Multi-Copy parallelism**

Данные входной модели разрезаются в соответствии с размерностью batchsize, таким образом, существующая форма с одной копией преобразуется в форму с несколькими копиями, так что нижележащий слой взаимодействует, в то время как другая копия выполняет вычислительную операцию без ожидания. Это гарантирует, что время вычислений и обмена данными нескольких копий дополняет друг друга для повышения производительности модели, в то время как разделение данных на несколько копий также уменьшает количество параметров входных данных операторов, тем самым сокращая время вычислений отдельных операторов, что значительно помогает повысить производительность модели.

![Multi-Copy](../pictures/Multi-copy_parallelism.png)

**Пояснение:** Multi-copy parallelism — это метод параллельных вычислений, используемый для ускорения обучения моделей, когда одна и та же модель копируется и выполняется на нескольких устройствах (GPU, TPU, или узлах кластера) одновременно, с различными наборами данных.

**Замечание:** Как я понял реализации в MindStore этих двух видов параллелизма нет

### Гибридный параллелизм

В настоящее время гибридные параллельные вычисления обычно используются следующие сценарии: гибридность оптимизатора, гибридность встраивания и гибридность PS (Parameter Server).

**Optimizer Heterogenetity**

Во время обучения большой модели состояние оптимизатора занимает большой объем памяти, что, в свою очередь, ограничивает размер модели, которую можно обучить. Оптимизатор и все его вычисления перебрасываются на CPU.

![Optimizer_heterogenenity](../pictures/Optimizer_heterogenenity.png)

**Embendding Heterogenetity**

В некоторых сетях, где необходимо проверять большие таблицы внедрения, размер таблиц встраивания часто составляет сотни гигабайт, что ограничено объемом памяти ускорителя и не может быть выполнено путем загрузки всей таблицы непосредственно на ускоритель. Поместив операторов, подключенных к таблице весов на процессоре для выполнения, мы избегаем проблемы, связанной с тем, что ускоритель не может обучить сеть из-за ограниченности памяти.

![Embeddings_heterogenetity](../pictures/Embeddings_heterogenetity.png)

**PS Heterogenetity**

**Parameter Server** — широко используемая архитектура в распределенном обучении, которая состоит из трех независимых компонентов: сервера, воркера и планировщика. Их функции заключаются в следующем:

**Сервер:** сохраняет веса модели и градиенты обратного вычисления и обновляет модель с использованием градиентов, отправленных рабочими.

**Воркер:** выполняет прямые и обратные вычисления в сети. Значение градиента для обратного вычисления загружается на сервер через API`Push`, а обновленная сервером модель загружается для воркера через API `Pull`.

**Планировщик:** устанавливает отношения связи между сервером и воркером.

Реализация: https://www.mindspore.cn/tutorials/experts/en/r2.3.1/parallel/parameter_server_training.html

![PS](../pictures/PS_heterogenetity.png)